
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql import types as T
from pyspark.sql import functions as F
import os
import configparser
from datetime import datetime
import shutil

### Step 1: Scope the Project and Gather Data

#### Scope 

This project is copy data from tourisms (in web) source and airport (in local)source then create table about tourisms. We can using these table we created to analyst information about tourisms. 
Example: How many tourist in each month in 2016...

#### Describe and Gather Data 
We using this data below for this project
1. file sas ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat 
- This data is tourisms from web. 

2. file airport-codes_csv.csv in local 
- table of airport codes and corresponding cities

3. file country-codes_csv.csv in local 
- table to explain country from code

4. file state-codes_csv.csv in local 
- table to explain state from code

5. file port-codes_csv.csv in local 
- table to explain port from code

In [2]:
# lOAD ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat data here
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#copy i94 data
df_i94 = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_i94.printSchema()
print(df_i94.count())
df_i94.limit(5).toPandas()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode i94addr  \
0    6.0  2016.0     4.0   692.0   692.0     XXX  20573.0      NaN    None   
1    7.0  2016.0     4.0   254.0   276.0     ATL  20551.0      1.0      AL   
2   15.0  2016.0     4.0   101.0   101.0     WAS  20545.0      1.0      MI   
3   16.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   
4   17.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   

   depdate   ...     entdepu  matflag  biryear   dtaddto gender insnum  \
0      NaN   ...           U     None   1979.0  10282016   None   None   
1      NaN   ...           Y     None   1991.0       D/S      M   None   
2  20691.0   ...        None        M   1961.0  09302016      M   None   
3  20567.0   ...        None        M   1988.0  09302016   None   None   
4  20567.0   ...        None        M   2012.0  09302016   None   None   

  airline        admnum  fltno visatype  
0    None  1.897628e+09   None       B2  
1    None  3.736796e+09  00296       F1  
2      OS  6.666432e+08     93       B2  
3      AA  9.246846e+10  00199       B2  
4      AA  9.246846e+10  00199       B2  

[5 rows x 28 columns]

In [3]:
#read data from airport-codes_csv
df_airports = spark.read.option("header",True).csv("input_data/airport-codes_csv.csv")
df_airports.printSchema()
print(df_airports.count())
df_airports.limit(5).toPandas()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

55075


ident           type                                name elevation_ft  \
0   00A       heliport                   Total Rf Heliport           11   
1  00AA  small_airport                Aero B Ranch Airport         3435   
2  00AK  small_airport                        Lowell Field          450   
3  00AL  small_airport                        Epps Airpark          820   
4  00AR         closed  Newport Hospital & Clinic Heliport          237   

  continent iso_country iso_region  municipality gps_code iata_code  \
0        NA          US      US-PA      Bensalem      00A      None   
1        NA          US      US-KS         Leoti     00AA      None   
2        NA          US      US-AK  Anchor Point     00AK      None   
3        NA          US      US-AL       Harvest     00AL      None   
4        NA          US      US-AR       Newport     None      None   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4       None                    -91.254898, 35.6087

In [4]:
#read data from country-codes_csv
df_country = spark.read.option("header",True).csv("input_data/country-codes_csv.csv")
df_country.printSchema()
print(df_country.count())
df_country.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

289


code                                               name
0  582  MEXICO Air Sea, and Not Reported (I-94, no lan...
1  236                                        AFGHANISTAN
2  101                                            ALBANIA
3  316                                            ALGERIA
4  102                                            ANDORRA

In [5]:
#read data from state-codes_csv
df_state = spark.read.option("header",True).csv("input_data/state-codes_csv.csv")
df_state.printSchema()
print(df_state.count())
df_state.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

55


code        name
0   AL     ALABAMA
1   AK      ALASKA
2   AZ     ARIZONA
3   AR    ARKANSAS
4   CA  CALIFORNIA

In [6]:
#read data from port-codes_csv
df_port = spark.read.option("header",True).csv("input_data/port-codes_csv.csv")
df_port.printSchema()
print(df_port.count())
df_port.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

660


code                          name
0  ALC                     ALCAN, AK
1  ANC                 ANCHORAGE, AK
2  BAR  BAKER AAF - BAKER ISLAND, AK
3  DAC             DALTONS CACHE, AK
4  PIZ    DEW STATION PT LAY DEW, AK

### Step 2: Explore and Assess the Data
#### Explore the Data 

1. file sas ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat or df_i94 table
- We change type some column of this data then drop duplicate data

2. file airport-codes_csv.csv in local or df_airports
- we drop duplicate data

3. file country-codes_csv.csv in local or df_country
- we drop duplicate data

4. file state-codes_csv.csv in local or df_state
- we drop duplicate data

5. file port-codes_csv.csv in local or df_port
- we drop duplicate data

#### Cleaning Steps
Document steps necessary to clean the data

In [7]:
# Performing cleaning tasks here
#change type of column of i94
df_i94 = df_i94 \
  .withColumn("cicid" ,
              df_i94["cicid"]
              .cast(T.IntegerType()))   \
  .withColumn("i94yr",
              df_i94["i94yr"]
              .cast(T.IntegerType()))    \
  .withColumn("i94mon"  ,
              df_i94["i94mon"]
              .cast(T.IntegerType())) \
  .withColumn("i94cit" ,
              df_i94["i94cit"]
              .cast(T.IntegerType()))   \
  .withColumn("i94res",
              df_i94["i94res"]
              .cast(T.IntegerType()))    \
  .withColumn("arrdate"  ,
              df_i94["arrdate"]
              .cast(T.TimestampType())) \
  .withColumn("depdate"  ,
              df_i94["depdate"]
              .cast(T.TimestampType())) \
  .withColumn("dtadfile"  ,
              df_i94["dtadfile"]
              .cast(T.TimestampType())) \
  .withColumn("biryear"  ,
              df_i94["biryear"]
              .cast(T.IntegerType())) \
  .withColumn("dtaddto"  ,
              df_i94["dtaddto"]
              .cast(T.TimestampType())) 

#drop duplicates
df_i94 = df_i94.drop_duplicates()

df_i94.printSchema()
print(df_i94.count())
df_i94.limit(5).toPandas()

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: timestamp (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: timestamp (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: timestamp (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: timestamp (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: d

cicid  i94yr  i94mon  i94cit  i94res i94port             arrdate  i94mode  \
0    238   2016       4     103     103     NYC 1970-01-01 05:42:25      1.0   
1    441   2016       4     103     103     MIA 1970-01-01 05:42:25      1.0   
2    556   2016       4     103     103     SFR 1970-01-01 05:42:25      1.0   
3    589   2016       4     103     103     TAM 1970-01-01 05:42:25      1.0   
4   1121   2016       4     104     104     NEW 1970-01-01 05:42:25      1.0   

  i94addr             depdate   ...     entdepu  matflag  biryear dtaddto  \
0      NY 1970-01-01 05:42:31   ...        None        M     1991    None   
1      FL 1970-01-01 05:42:33   ...        None        M     1983    None   
2      CA 1970-01-01 05:42:26   ...        None        M     1974    None   
3      FL 1970-01-01 05:42:54   ...        None        M     1986    None   
4      NY 1970-01-01 05:42:35   ...        None        M     1978    None   

  gender insnum airline        admnum  fltno visatype  
0      F   None      AB  5.542168e+10  07450       WT  
1   None   None      OS  5.542591e+10  00097       WT  
2      M   None      LH  5.546223e+10  00458       WB  
3      F   None      LH  5.545721e+10  00482       WT  
4      M   None      UA  6.652610e+08     56       WT  

[5 rows x 28 columns]

In [8]:
#change type of column of df_airports

#drop duplicates
df_airports = df_airports.drop_duplicates()

df_airports.printSchema()
print(df_airports.count())
df_airports.limit(5).toPandas()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

55075


ident           type                                       name  \
0  01AK       heliport  Providence Seward Medical Center Heliport   
1  04WA  small_airport                         Ox Meadows Airport   
2  08OR  small_airport                        Saxon Sycan Airport   
3  0CD9       heliport              Swedish S.W. Medical Heliport   
4  0GA3  small_airport                          Ayresouth Airport   

  elevation_ft continent iso_country iso_region municipality gps_code  \
0          120        NA          US      US-AK       Seward     01AK   
1         2345        NA          US      US-WA      Spokane     04WA   
2         4990        NA          US      US-OR  Silver Lake     08OR   
3         5563        NA          US      US-CO    Littleton     0CD9   
4         1287        NA          US      US-GA       Temple     0GA3   

  iata_code local_code                              coordinates  
0      None       01AK       -149.446249008, 60.105873975399994  
1      None       04WA             -117.43699646, 47.5817985535  
2      None       08OR  -121.11699676513672, 42.839298248291016  
3      None       0CD9             -105.091003418, 39.603099823  
4      None       0GA3    -85.06079864501953, 33.77009963989258

In [9]:
#change type of column of df_country

#drop duplicates
df_country = df_country.drop_duplicates()

df_country.printSchema()
print(df_country.count())
df_country.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

289


code                                             name
0  373                                     SOUTH AFRICA
1  206                                        HONG KONG
2  214                                            MACAU
3  739  INVALID: DRONNING MAUD LAND (ANTARCTICA-NORWAY)
4  190                            No Country Code (190)

In [10]:
#change type of column of df_state

#drop duplicates
df_state = df_state.drop_duplicates()

df_state.printSchema()
print(df_state.count())
df_state.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

55


code       name
0   WY    WYOMING
1   TN  TENNESSEE
2   KS     KANSAS
3   IA       IOWA
4   NV     NEVADA

In [11]:
#change type of column of df_port

#drop duplicates
df_port = df_port.drop_duplicates()

df_port.printSchema()
print(df_port.count())
df_port.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)

660


code                         name
0  HOM                    HOMER, AK
1  WAL                 WALHALLA, ND
2  MRR               MORRISTOWN, NY
3  IND                   LOS INDIOS
4  BED  HANSCOM FIELD - BEDFORD, MA

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

We have tourisms table with lot of information
airlines, time, country_code, state_code, port_code table additional information for tourisms table.

So we use star schema for this project.

The schema is below

![MarineGEO]( image/diagram.PNG "")

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model


##### Fact Table
1. tourisms - records i94 
- airport_id,type,name,iso_country,iso_region,iata_code,local_code,


##### Dimension Tables

1. airport:  table of airport codes and corresponding cities
- airport_id,type,name,iso_country,iso_region,iata_code,local_code

2. time: timestamps of tourisms for travals
- start_time, hour, day, week, month, year, weekday

3. port_code: table to explain port from code
- name, code

4. state_code: table to explain state from code
- name, code

5. country_code: table to explain country from code
- name, code


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
- tourisms from df_i94 data
- airlines from df_airports data
- time from df_i94 data
- country_code from country_code
- port_code from port_code
- state_code from state_code

![MarineGEO]( image/tourisms.PNG "")

![MarineGEO]( image/airporttime.PNG "")

![MarineGEO]( image/countrystatecode.PNG "")

In [12]:
# extract columns to create tourisms table
tourisms_table = df_i94.select(F.col("fltno").alias("flights_id"),
                            F.col("airline").alias("airlines_id"),
                            F.col("arrdate").alias("arrival_date"),
                            F.col("depdate").alias("departure_date"),
                            F.col("entdepa").alias("arrival_flag"),
                            F.col("entdepd").alias("departure_flag"),
                            F.col("entdepu").alias("update_flag"),
                            F.col("matflag").alias("match_flag"),
                            F.col("gender").alias("gender"),
                            F.col("i94mode").alias("model"),
                            F.col("i94addr").alias("state"),
                            F.col("i94port").alias("port"),
                            F.col("i94cit").alias("country")
                            )
# remove the duplicate records
tourisms_table = tourisms_table.drop_duplicates()

In [13]:
# get_datetime = udf()
df_time = df_i94
df_time = df_i94.withColumn("hour", F.hour("depdate"))
df_time = df_time.withColumn("day", F.dayofmonth("depdate"))
df_time = df_time.withColumn("week", F.weekofyear("depdate"))
df_time = df_time.withColumn("month", F.month("depdate"))
df_time = df_time.withColumn("year", F.year("depdate"))
df_time = df_time.withColumn("weekday", F.dayofweek("depdate"))

# extract columns to create time table
time_table = df_time.select(F.col("depdate").alias("start_time"),
                      F.col("hour"),
                      F.col("day"),
                      F.col("week"),
                      F.col("month"),
                      F.col("year"),
                      F.col("weekday")
                     )

# remove the duplicate records
time_table = time_table.drop_duplicates()

In [14]:
# create new column
df_airports = df_airports.withColumn("airport_id",
                  F.col("ident").substr(-2,2))

# extract columns for users table    
airport_table = df_airports.select(F.col("airport_id"),
                        F.col("type"),
                        F.col("name"),
                        F.col("iso_country"),
                        F.col("iso_region"),
                        F.col("iata_code"),
                        F.col("local_code")
                       )

# remove the duplicate records
airport_table = airport_table.drop_duplicates()

In [15]:
# extract columns for port table    
port_table = df_port.select(F.col("code"),
                        F.col("name"))

# remove the duplicate records
port_table = port_table.drop_duplicates()

In [16]:
# extract columns for state table    
state_table = df_state.select(F.col("code"),
                        F.col("name"))

# remove the duplicate records
state_table = state_table.drop_duplicates()

In [17]:
# extract columns for country table    
country_table = df_country.select(F.col("code"),
                        F.col("name"))

# remove the duplicate records
country_table = country_table.drop_duplicates()

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [18]:
#function delete parquet
def delete_parquet_file(output_data):
    array_table = ["tourisms_table", "time_table","airport_table","port_codes_table", "country_codes_table", "state_codes_table"]
    for li in array_table:
        if os.path.exists(output_data+ li + ".parquet"):
            shutil.rmtree(output_data+ li + ".parquet")
            

#delete all file parquet existe
delete_parquet_file("output_data/")

In [19]:
#write to parquet
# write tourisms table to parquet files 
output_data = "output_data/"
tourisms_table.write.parquet(output_data+"tourisms_table.parquet")

# write time table to parquet files partitioned by year and month
time_table.write.parquet(output_data+"time_table.parquet")

# write users table to parquet files
airport_table.write.parquet(output_data+"airport_table.parquet")

# write port table to parquet files
port_table.write.parquet(output_data+"port_codes_table.parquet")

# write state table to parquet files
state_table.write.parquet(output_data+"state_codes_table.parquet")

# write country table to parquet files
country_table.write.parquet(output_data+"country_codes_table.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [20]:
def check_data_table(spark, output_data, table ):
    if os.path.exists(output_data+ table + ".parquet"):
        check=spark.read.parquet(output_data+ table + ".parquet")
        if (check.count() > 0):
            return check.count()
        else:
            return "No"
    else:
        return "No"

In [21]:

def check_data(spark, output_data):
    rows = []
    array_table = ["tourisms_table", "time_table","airport_table","port_codes_table", "country_codes_table", "state_codes_table"]
    for li in array_table:
        check = check_data_table(spark, output_data, li)
        rows.append([li, check])
    columns = ['table', 'check']
    df_check_data = spark.createDataFrame(rows, columns)
    return df_check_data

In [22]:
df_check_data = check_data(spark,"output_data/")

In [23]:
df_check_data.printSchema()
print(df_check_data.count())
df_check_data.toPandas()

root
 |-- table: string (nullable = true)
 |-- check: long (nullable = true)

6


table    check
0       tourisms_table  1931005
1           time_table      236
2        airport_table    55074
3     port_codes_table      660
4  country_codes_table      289
5    state_codes_table       55

#### 4.3 Data dictionary 
Create a data dictionary for data model as below

![MarineGEO]( image/dict1.PNG "")

![MarineGEO]( image/dict2.PNG "")

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

1. Clearly state the rationale for the choice of tools and technologies for the project.
 - Scope the Project and Gather Data below
     * file sas ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat in site
     * file airport-codes_csv.csv in local
     * file port-codes_csv.csv in local
     * file state-codes_csv.csv in local
     * file country-codes_csv.csv in local

 - Explore and Assess the Data: Cleaning data, 
 - Mapping data: mapping and  filling data 
 - Then checking data quality

  * So we using PySpark and Apache Spark for this project. As we know, spark support a lot of for user. 
    * It easy to use :  Spark can handle multiple petabytes of clustered data of more than 8000 nodes at a time, so it very speed.
    * It is very speed : Spark can handle multiple petabytes of clustered data of more than 8000 nodes at a time
    * it 
  * So we using Star schema for this project
    * Star schema quickly provide query responses
    * It provides a flexible design, can changed easily or added as the database grows
    * provides a parallel in design
    * star schema is simple to understand and navigate



2. Propose how often the data should be updated and why.
 -  I think data can update monthly, because i94 data tourisms update monthly and orther data local are added to clarify tourisms data. So we can update monthly data


3. Write a description of how you would approach the problem differently under the following scenarios:
 - The data was increased by 100x :  
     * We can using Parallel Processing in Python with AWS Lambda, it excecute lot of process in same time. So we can save time.
     
 - The data populates a dashboard that must be updated on a daily basis by 7am every day :
     * Using task scheduler window and using Virtual machine for this scheduler window
     * Using Apache Airflow, we can schedule task and check data also
     
 - The database needed to be accessed by 100+ people :
     *  Usinf EC2 instances (this using Elastic Load Balancing (ELB)). ELB is a redundant, horizontally scaled service that requires zero administration from my end and supports SSL termination and sticky sessions.
     * for database, add redundancy by using the Multi-AZ feature of RDS